In [1]:
import numpy as np
import tensorflow as tf
import glob
from pydub import AudioSegment
from audioset import vggish_keras_embeddings
from time import time
import utils
import youtube_dl
import pandas as pd
import h5py
import os
import datetime

%matplotlib inline

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from '/home/oppapie/projects/LaughDetection/utils.py'>

In [4]:
#url = 'https://www.youtube.com/watch?v=fpf1IYU0poY' # Trump state of the union 2019
#url = 'https://www.youtube.com/watch?v=Ymwe4DB_HsU'
#url = 'https://www.youtube.com/watch?v=vP4iY1TtS3s' # I have a dream speech
#url = 'https://www.youtube.com/watch?v=-taEc2havwA' # jim carrey speech
#url = 'https://www.youtube.com/watch?v=KRGgQP47Vgw'
#url = 'https://www.youtube.com/watch?v=k0jJL_YFyIU'
url = 'https://www.youtube.com/watch?v=W_GKJAF4c3g' # Random tennis video



save_folder = 'sample_inference/'
def download_audio(url, download=True, save_folder=save_folder):

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'writesubtitles': True,
        'writeautomaticsub': True,
        'quiet': True,
        'outtmpl': save_folder+'%(title)s-%(id)s.%(ext)s',
        #'progress_hooks': [my_hook],
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=download)  # Download into the current working directory
        filename = ydl.prepare_filename(result)
    return filename
filename = download_audio(url)


In [6]:
filename

'sample_inference/Funniest Moments & Fails - Monte-Carlo 2019-W_GKJAF4c3g.webm'

In [7]:
clipsize = 1000
savename=filename+'_clipsize'+str(clipsize)
start = time()
def embed_audio(filename, clipsize=clipsize, savename=filename+'_clipsize'+str(clipsize)):
    if savename[-3:] != '.h5':
        savename = savename + '.h5'
    exists = os.path.isfile(savename)
    if exists:
        print('Embedding file detected.')
        print('Loading embedding file:', savename)
        f = h5py.File(savename, 'r')
        return f.items()
    else:
        print('Embedding file not detected.')
        print('Running embedder on audio.')
        list_of_embeddings = []
        audio_embedder = vggish_keras_embeddings.VGGishEmbedder(None)
        for idx, val in enumerate(utils.sound_slice_generator(filename, clipsize=clipsize, sample_rate=44100)):
            waveform, sample_rate = val[0], val[1]
            if val[0].shape[0] < sample_rate:
                waveform = np.pad(val[0], (0, sample_rate - val[0].shape[0]), mode='constant')
            processed_embedding =  audio_embedder.convert_waveform_to_embedding(waveform, sample_rate)

            embedding_final = np.expand_dims(processed_embedding, axis=0)
            list_of_embeddings.append((idx, embedding_final))
            if idx != 0 and idx % 100 == 0:
                print(idx, "samples embedded")
        utils.save_embeddings_hdf(savename, list_of_embeddings)
        print('Saved embeddings to:', savename)
        return list_of_embeddings

predictions = []

model = tf.keras.models.load_model('Models/v2_cheer_applause_LSTM_ThreeLayer_100Epochs.h5')

for idx, v in embed_audio(filename, clipsize=clipsize):
    predictions.append((idx, model.predict(v)))
    
end = time()
print('\nFinished.\n')
print('Elapsed Time:', f'{end - start:.2f}', 'seconds')

W0423 15:27:03.535386 140495682406208 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fc66c5795f8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0423 15:27:03.793719 140495682406208 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fc66c34c470>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0423 15:27:03.926867 140495682406208 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fc67c54f518>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Embedding file not detected.
Running embedder on audio.
No of sound slices 404.7368253968254
100 samples embedded
200 samples embedded
300 samples embedded
400 samples embedded
Saved embeddings to: sample_inference/Funniest Moments & Fails - Monte-Carlo 2019-W_GKJAF4c3g.webm_clipsize1000.h5

Finished.

Elapsed Time: 18.96 seconds


## Saving predictions to csv

In [8]:
x = [datetime.timedelta(seconds=float(i[0])/(2000/clipsize)) for i in predictions]
y = [i[1][0][0] for i in predictions]

df = pd.DataFrame({'timestamp': x, 'prediction': y})
df.to_csv(savename+'_predictions.csv', index=False)
print(savename+'_predictions.csv')

sample_inference/Funniest Moments & Fails - Monte-Carlo 2019-W_GKJAF4c3g.webm_clipsize1000_predictions.csv


### Continue to Explore_Subtitles_vs_Predictions.ipynb